<a href="https://colab.research.google.com/github/rzbsys/CNN_MODEL/blob/main/%ED%94%8C%EB%9D%BC%EC%8A%A4%ED%8B%B1_%EC%9D%B8%EC%8B%9D%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Keras로 플라스틱이 재활용 가능한지 인식 가능한 모델 생성하기**

20529 전민국

CTRL+ENTER = 선택한 셀의 코드 실행

##**0. GPU 확인**


GPU가 연결되어있는지 확인하는 코드

P100 = 더 좋은거 | 
tesla T4 = 좋은거 | 
tesla k80 = 덜 좋은거 

보통 최대 W수가 높으면 좋고, 좋은거 걸릴때 까지 런타임 초기화를 진행하는 것을 추천함

In [ ]:
!nvidia-smi

##**1. 필수 라이브러리 import**

필요한 라이브러리를 import 해준다

*   Tensorflow
*   Keras
*   Glob
*   Sckit-learn
*   OpenCV
*   Numpy
*   Matplot


In [2]:
import tensorflow as tf
from tensorflow import keras
from google.colab import files
from glob import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt

##**2. 데이터셋 다운로드**


우선, 필요한 모듈을 모두 import 해준후 Kaggle의 데이터셋을 다운로드 한다.

[Kaggle 원본 링크](https://www.kaggle.com/piaoya/plastic-recycling-codes)

**Kaggle API Token json 다운로드 방법**

1. [Kaggle](https://kaggle.com) 사이트에 접속하여 로그인
2. 오른쪽 상단에 있는 프로필 사진 클릭
3. Account 클릭
4. API에 있는 Create New API Token 버튼 클릭


In [ ]:
files.upload()
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d piaoya/plastic-recycling-codes
!unzip plastic-recycling-codes.zip

##**3. 이미지 전처리**

img = 이미지 저장

label = 저장된 이미지 순서대로 이미지의 라벨 저장



1.   cv2.imread로 불러오기
2.   cv2.resize로 모델에 맞추어 크기 바꾸기
3.   cv2.normalize로 이미지 정규화 진행
4.   Numpy Array로 변환


In [ ]:
from keras.preprocessing.image import load_img, img_to_array
img = []
label = []
for i in glob('/content/seven_plastics/*/*.jpg'):
  img += [cv2.normalize(cv2.resize(cv2.imread(i), (224, 224)), None, 0, 255, cv2.NORM_MINMAX)]
  label += i.split('/')[-2][0]
img = np.asarray(img, dtype=np.float32) / 255.0
label = np.asarray(label, dtype=np.int32)

print(img.shape, label.shape)
plt.imshow(img[1])

##**4. 데이터셋 분리**

Sckit-learn의 train_test_split를 활용하여 데이터셋을 나눔.

Test 데이터셋의 비율은 10%만을 사용

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(img, label, random_state=1500, test_size=0.1)
print(len(X_train), len(Y_train), len(X_test), len(Y_test))
print(type(X_train), type(Y_train))

Matplot Lib를 통해 분류된 데이터를 확인

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i])
    plt.xlabel(Y_train[i])
plt.show()

##**5. 모델 구성**

원하는 모델 하나를 골라 선택

###ResNet50V2 모델

In [68]:
from tensorflow.keras.applications import ResNet50V2
model = ResNet50V2(weights='imagenet')

###VGG16 모델

In [ ]:
from tensorflow.keras.applications import VGG16
model = VGG16(weights='imagenet')

###MobileNet 모델

In [ ]:
from tensorflow.keras.applications import MobileNetV3Small
model = MobileNetV3Small(weights='imagenet')

###Xception 모델

In [19]:
from tensorflow.keras.applications import Xception
model = Xception(weights='imagenet')

###Inception 모델

In [ ]:
from tensorflow.keras.applications import InceptionV3
model = InceptionV3(weights='imagenet')

##**6. 모델 학습 진행**

텐서보드 콜백 정의

In [ ]:
#선택사항
%load_ext tensorboard
tensorboard_callback = keras.callbacks.TensorBoard(log_dir='./log', write_graph=True, histogram_freq=True)

모델 학습 진행


*   Loss Function = sparse_categorical_crossentropy
*   Optimizer = Adam



In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(X_train, Y_train, batch_size=30, epochs=100, validation_data=(X_test, Y_test)
#,callbacks=tensorboard_callback #선택사항
)

학습 진행 과정을 그래프로 표현

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Accuracy')
plt.legend()
plt.figure() 
plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Loss')
plt.legend()
plt.show()

모델 저장하기

In [ ]:
model.save('End.h5')

##**7. 실제 이미지로 예측**

코드 실행 후 파일 업로드

In [ ]:
file_name = files.upload()
file_name = file_name.keys()
file_name
for i in file_name:
  file_name += i

Custom_Img = [cv2.normalize(cv2.resize(cv2.imread('./' + file_name), (224, 224)), None, 0, 255, cv2.NORM_MINMAX)]
Custom_Img = np.asarray(img, dtype=np.float32) / 255.0
Pre_data = model.predict(Custom_Img)
print(Pre_data.argmax())